In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import random

# Setting random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)  # Set seed

data = pd.read_csv('final_50_all.csv')

num_features = ['temperature_2m (°C)', 'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_100m (km/h)']

# Feature engineering
data['hour'] = pd.to_datetime(data['hour'])
data['year'] = data['hour'].dt.year
data['month'] = data['hour'].dt.month
data['day'] = data['hour'].dt.day
data['hour_of_day'] = data['hour'].dt.hour
data['weekday'] = data['hour'].dt.weekday

data['hour_sin'] = np.sin(2 * np.pi * data['hour_of_day'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour_of_day'] / 24)
data['weekday_sin'] = np.sin(2 * np.pi * data['weekday'] / 7)
data['weekday_cos'] = np.cos(2 * np.pi * data['weekday'] / 7)
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

# Scaling numerical features
scaler = MinMaxScaler()
data[num_features] = scaler.fit_transform(data[num_features])
data.drop(['month','day','hour', 'hour_of_day', 'weekday'], axis=1, inplace=True)

# Extracting features and target
features = num_features + ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos','month_sin','month_cos']
target = 'ride_count'

X = data[features].values
y = data[target].values

# Normalizing data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Create sequences for LSTM input
def create_sequences(data, target, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(target[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 24  # Using the past 24 hours of data to predict
X, y = create_sequences(X, y, sequence_length)

# Implementing KFold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Loaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

    # Define the model
    class GRUModelWithSimpleAttention(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, dropout=0.2, attention_dropout=0.1):
            super(GRUModelWithSimpleAttention, self).__init__()
            self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
            self.attention_score = nn.Linear(hidden_size, 1)
            self.attention_dropout = nn.Dropout(attention_dropout)
            self.fc = nn.Linear(hidden_size, output_size)
            self.dropout = nn.Dropout(dropout)

        def forward(self, x):
            gru_out, _ = self.gru(x)
            attn_scores = self.attention_score(gru_out)
            attn_weights = torch.softmax(attn_scores, dim=1)
            attn_weights = self.attention_dropout(attn_weights)
            context_vector = torch.sum(attn_weights * gru_out, dim=1)
            out = self.dropout(context_vector)
            out = self.fc(out)
            return out

    # Initialize model, loss, and optimizer
    model = GRUModelWithSimpleAttention(input_size=len(features), hidden_size=64, output_size=1, dropout=0.2, attention_dropout=0.1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    best_val_loss = float('inf')
    patience, trials = 10, 0

    for epoch in range(50):  # Reduced number of epochs for brevity
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch).squeeze()
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Testing model
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                y_pred = model(X_batch).squeeze()
                test_loss += criterion(y_pred, y_batch).item()

        test_loss /= len(test_loader)
        print(f"Fold {fold+1}, Epoch {epoch+1}: Train Loss {train_loss:.4f}, Test Loss {test_loss:.4f}")

        if test_loss < best_val_loss:
            best_val_loss = test_loss
            trials = 0
            torch.save(model.state_dict(), f'model_fold_{fold+1}.pth')
        else:
            trials += 1
            if trials >= patience:
                print("Early stopping...")
                break

    # Load the best model and evaluate on the test set
    model.load_state_dict(torch.load(f'model_fold_{fold+1}.pth'))
    model.eval()
    y_preds, y_trues = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch).squeeze()
            y_preds.extend(y_pred.tolist())
            y_trues.extend(y_batch.tolist())

    mae = mean_absolute_error(y_trues, y_preds)
    rmse = np.sqrt(mean_squared_error(y_trues, y_preds))
    results.append((mae, rmse))
    print(f"Fold {fold+1} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# Calculate average MAE and RMSE across folds
avg_mae = np.mean([res[0] for res in results])
avg_rmse = np.mean([res[1] for res in results])
print(f"\nAverage MAE: {avg_mae:.4f}, Average RMSE: {avg_rmse:.4f}")


Fold 1, Epoch 1: Train Loss 243.7425, Test Loss 204.0431
Fold 1, Epoch 2: Train Loss 167.2036, Test Loss 102.4151
Fold 1, Epoch 3: Train Loss 105.4399, Test Loss 96.0960
Fold 1, Epoch 4: Train Loss 102.3840, Test Loss 93.1141
Fold 1, Epoch 5: Train Loss 99.4453, Test Loss 92.4906
Fold 1, Epoch 6: Train Loss 97.0529, Test Loss 86.1986
Fold 1, Epoch 7: Train Loss 91.7267, Test Loss 80.1030
Fold 1, Epoch 8: Train Loss 86.5610, Test Loss 79.8065
Fold 1, Epoch 9: Train Loss 84.4327, Test Loss 75.0851
Fold 1, Epoch 10: Train Loss 83.5114, Test Loss 73.8982
Fold 1, Epoch 11: Train Loss 81.1430, Test Loss 72.4198
Fold 1, Epoch 12: Train Loss 80.9970, Test Loss 72.5339
Fold 1, Epoch 13: Train Loss 79.4513, Test Loss 70.7466
Fold 1, Epoch 14: Train Loss 79.9100, Test Loss 73.1786
Fold 1, Epoch 15: Train Loss 78.9962, Test Loss 70.2452
Fold 1, Epoch 16: Train Loss 78.6866, Test Loss 68.9847
Fold 1, Epoch 17: Train Loss 77.0082, Test Loss 70.3501
Fold 1, Epoch 18: Train Loss 77.3292, Test Loss 69.

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/1834357499.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 1 - MAE: 5.5022, RMSE: 7.9393
Fold 2, Epoch 1: Train Loss 244.8329, Test Loss 213.6348
Fold 2, Epoch 2: Train Loss 172.5303, Test Loss 114.3899
Fold 2, Epoch 3: Train Loss 104.6130, Test Loss 103.3774
Fold 2, Epoch 4: Train Loss 101.3671, Test Loss 101.3274
Fold 2, Epoch 5: Train Loss 99.9052, Test Loss 100.4946
Fold 2, Epoch 6: Train Loss 98.1777, Test Loss 98.0138
Fold 2, Epoch 7: Train Loss 96.4765, Test Loss 96.6921
Fold 2, Epoch 8: Train Loss 95.0920, Test Loss 92.2000
Fold 2, Epoch 9: Train Loss 89.8422, Test Loss 85.8883
Fold 2, Epoch 10: Train Loss 85.4669, Test Loss 80.5623
Fold 2, Epoch 11: Train Loss 82.1017, Test Loss 78.8288
Fold 2, Epoch 12: Train Loss 79.6705, Test Loss 77.9753
Fold 2, Epoch 13: Train Loss 79.2478, Test Loss 79.2422
Fold 2, Epoch 14: Train Loss 78.1880, Test Loss 80.8158
Fold 2, Epoch 15: Train Loss 78.0260, Test Loss 74.8930
Fold 2, Epoch 16: Train Loss 77.3461, Test Loss 77.6935
Fold 2, Epoch 17: Train Loss 76.4720, Test Loss 74.7878
Fold 2, Epoch

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/1834357499.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 3, Epoch 1: Train Loss 242.9917, Test Loss 203.4605
Fold 3, Epoch 2: Train Loss 179.4683, Test Loss 107.3719
Fold 3, Epoch 3: Train Loss 105.7649, Test Loss 97.0190
Fold 3, Epoch 4: Train Loss 101.4395, Test Loss 93.5526
Fold 3, Epoch 5: Train Loss 98.6810, Test Loss 90.8478
Fold 3, Epoch 6: Train Loss 95.1589, Test Loss 85.9514
Fold 3, Epoch 7: Train Loss 88.8612, Test Loss 78.5959
Fold 3, Epoch 8: Train Loss 85.2375, Test Loss 75.0162
Fold 3, Epoch 9: Train Loss 83.6448, Test Loss 75.2276
Fold 3, Epoch 10: Train Loss 80.8715, Test Loss 72.8702
Fold 3, Epoch 11: Train Loss 80.4238, Test Loss 71.3618
Fold 3, Epoch 12: Train Loss 80.0994, Test Loss 72.0677
Fold 3, Epoch 13: Train Loss 78.3625, Test Loss 69.7504
Fold 3, Epoch 14: Train Loss 78.2071, Test Loss 69.2664
Fold 3, Epoch 15: Train Loss 78.0431, Test Loss 70.2540
Fold 3, Epoch 16: Train Loss 77.6895, Test Loss 69.5682
Fold 3, Epoch 17: Train Loss 76.8096, Test Loss 69.8792
Fold 3, Epoch 18: Train Loss 76.7081, Test Loss 68.

...

Fold 4, Epoch 1: Train Loss 243.0483, Test Loss 205.9942
Fold 4, Epoch 2: Train Loss 180.6908, Test Loss 108.8755
Fold 4, Epoch 3: Train Loss 103.1256, Test Loss 100.0639
Fold 4, Epoch 4: Train Loss 99.0422, Test Loss 97.5285
Fold 4, Epoch 5: Train Loss 96.8618, Test Loss 95.2063
Fold 4, Epoch 6: Train Loss 94.5352, Test Loss 91.9662
Fold 4, Epoch 7: Train Loss 91.3491, Test Loss 89.2233
Fold 4, Epoch 8: Train Loss 87.0911, Test Loss 83.4508
Fold 4, Epoch 9: Train Loss 83.2681, Test Loss 83.4685
Fold 4, Epoch 10: Train Loss 81.7285, Test Loss 79.5072
Fold 4, Epoch 11: Train Loss 80.5225, Test Loss 79.4360
Fold 4, Epoch 12: Train Loss 78.8539, Test Loss 77.5522
Fold 4, Epoch 13: Train Loss 78.4052, Test Loss 82.4890
Fold 4, Epoch 14: Train Loss 77.2487, Test Loss 75.5562
Fold 4, Epoch 15: Train Loss 76.1423, Test Loss 75.7116
Fold 4, Epoch 16: Train Loss 75.8263, Test Loss 81.9812
Fold 4, Epoch 17: Train Loss 75.3391, Test Loss 73.7676
Fold 4, Epoch 18: Train Loss 75.0243, Test Loss 74.

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/1834357499.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 5, Epoch 1: Train Loss 242.7775, Test Loss 204.6353
Fold 5, Epoch 2: Train Loss 174.9928, Test Loss 104.6854
Fold 5, Epoch 3: Train Loss 105.8649, Test Loss 95.8758
Fold 5, Epoch 4: Train Loss 102.6593, Test Loss 92.7744
Fold 5, Epoch 5: Train Loss 100.6980, Test Loss 91.1566
Fold 5, Epoch 6: Train Loss 99.4794, Test Loss 93.4726
Fold 5, Epoch 7: Train Loss 97.8100, Test Loss 84.3961
Fold 5, Epoch 8: Train Loss 88.9980, Test Loss 75.7261
Fold 5, Epoch 9: Train Loss 84.1075, Test Loss 77.1799
Fold 5, Epoch 10: Train Loss 81.7014, Test Loss 73.3711
Fold 5, Epoch 11: Train Loss 81.1437, Test Loss 72.5707
Fold 5, Epoch 12: Train Loss 80.1805, Test Loss 72.0784
Fold 5, Epoch 13: Train Loss 78.9102, Test Loss 71.2846
Fold 5, Epoch 14: Train Loss 78.8404, Test Loss 73.4162
Fold 5, Epoch 15: Train Loss 78.0587, Test Loss 69.8796
Fold 5, Epoch 16: Train Loss 77.7573, Test Loss 78.4086
Fold 5, Epoch 17: Train Loss 78.2069, Test Loss 71.8798
Fold 5, Epoch 18: Train Loss 77.6299, Test Loss 69

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/1834357499.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 5 - MAE: 5.5868, RMSE: 8.0587

Average MAE: 5.6075, Average RMSE: 8.1109


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import random

# Setting random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)  # Set seed

data = pd.read_csv('final_50_all.csv')

num_features = ['temperature_2m (°C)', 'apparent_temperature (°C)', 'rain (mm)', 'wind_speed_100m (km/h)']

# Feature engineering
data['hour'] = pd.to_datetime(data['hour'])
data['year'] = data['hour'].dt.year
data['month'] = data['hour'].dt.month
data['day'] = data['hour'].dt.day
data['hour_of_day'] = data['hour'].dt.hour
data['weekday'] = data['hour'].dt.weekday

data['hour_sin'] = np.sin(2 * np.pi * data['hour_of_day'] / 24)
data['hour_cos'] = np.cos(2 * np.pi * data['hour_of_day'] / 24)
data['weekday_sin'] = np.sin(2 * np.pi * data['weekday'] / 7)
data['weekday_cos'] = np.cos(2 * np.pi * data['weekday'] / 7)
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

# Scaling numerical features
scaler = MinMaxScaler()
data[num_features] = scaler.fit_transform(data[num_features])
data.drop(['month','day','hour', 'hour_of_day', 'weekday'], axis=1, inplace=True)

# Extracting features and target
features = num_features + ['hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos','month_sin','month_cos']
target = 'ride_count'

X = data[features].values
y = data[target].values

# Normalizing data
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

# Create sequences for LSTM input
def create_sequences(data, target, sequence_length):
    X, y = [], []
    for i in range(len(data) - sequence_length):
        X.append(data[i:i + sequence_length])
        y.append(target[i + sequence_length])
    return np.array(X), np.array(y)

sequence_length = 48  # Using the past 24 hours of data to predict
X, y = create_sequences(X, y, sequence_length)

# Implementing KFold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = []

for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Convert to tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Loaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
    test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

    # Define the model
    class GRUModelWithSimpleAttention(nn.Module):
        def __init__(self, input_size, hidden_size, output_size, dropout=0.2, attention_dropout=0.1):
            super(GRUModelWithSimpleAttention, self).__init__()
            self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
            self.attention_score = nn.Linear(hidden_size, 1)
            self.attention_dropout = nn.Dropout(attention_dropout)
            self.fc = nn.Linear(hidden_size, output_size)
            self.dropout = nn.Dropout(dropout)

        def forward(self, x):
            gru_out, _ = self.gru(x)
            attn_scores = self.attention_score(gru_out)
            attn_weights = torch.softmax(attn_scores, dim=1)
            attn_weights = self.attention_dropout(attn_weights)
            context_vector = torch.sum(attn_weights * gru_out, dim=1)
            out = self.dropout(context_vector)
            out = self.fc(out)
            return out

    # Initialize model, loss, and optimizer
    model = GRUModelWithSimpleAttention(input_size=len(features), hidden_size=64, output_size=1, dropout=0.2, attention_dropout=0.1)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    best_val_loss = float('inf')
    patience, trials = 10, 0

    for epoch in range(50):  # Reduced number of epochs for brevity
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            y_pred = model(X_batch).squeeze()
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Testing model
        model.eval()
        test_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                y_pred = model(X_batch).squeeze()
                test_loss += criterion(y_pred, y_batch).item()

        test_loss /= len(test_loader)
        print(f"Fold {fold+1}, Epoch {epoch+1}: Train Loss {train_loss:.4f}, Test Loss {test_loss:.4f}")

        if test_loss < best_val_loss:
            best_val_loss = test_loss
            trials = 0
            torch.save(model.state_dict(), f'model_fold_{fold+1}.pth')
        else:
            trials += 1
            if trials >= patience:
                print("Early stopping...")
                break

    # Load the best model and evaluate on the test set
    model.load_state_dict(torch.load(f'model_fold_{fold+1}.pth'))
    model.eval()
    y_preds, y_trues = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch).squeeze()
            y_preds.extend(y_pred.tolist())
            y_trues.extend(y_batch.tolist())

    mae = mean_absolute_error(y_trues, y_preds)
    rmse = np.sqrt(mean_squared_error(y_trues, y_preds))
    results.append((mae, rmse))
    print(f"Fold {fold+1} - MAE: {mae:.4f}, RMSE: {rmse:.4f}")

# Calculate average MAE and RMSE across folds
avg_mae = np.mean([res[0] for res in results])
avg_rmse = np.mean([res[1] for res in results])
print(f"\nAverage MAE: {avg_mae:.4f}, Average RMSE: {avg_rmse:.4f}")


Fold 1, Epoch 1: Train Loss 246.5969, Test Loss 197.8791
Fold 1, Epoch 2: Train Loss 194.1516, Test Loss 128.5456
Fold 1, Epoch 3: Train Loss 126.6488, Test Loss 108.9884
Fold 1, Epoch 4: Train Loss 120.4667, Test Loss 106.2424
Fold 1, Epoch 5: Train Loss 117.9492, Test Loss 104.3088
Fold 1, Epoch 6: Train Loss 116.3518, Test Loss 102.8310
Fold 1, Epoch 7: Train Loss 115.0033, Test Loss 101.3728
Fold 1, Epoch 8: Train Loss 113.6009, Test Loss 99.6148
Fold 1, Epoch 9: Train Loss 111.9644, Test Loss 96.8459
Fold 1, Epoch 10: Train Loss 108.4578, Test Loss 91.8248
Fold 1, Epoch 11: Train Loss 101.9110, Test Loss 80.6201
Fold 1, Epoch 12: Train Loss 89.8171, Test Loss 66.8473
Fold 1, Epoch 13: Train Loss 81.5825, Test Loss 62.5950
Fold 1, Epoch 14: Train Loss 79.3319, Test Loss 61.6651
Fold 1, Epoch 15: Train Loss 77.4616, Test Loss 59.5131
Fold 1, Epoch 16: Train Loss 76.0083, Test Loss 61.3381
Fold 1, Epoch 17: Train Loss 75.6368, Test Loss 61.4862
Fold 1, Epoch 18: Train Loss 75.3172, T

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/2435093727.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 1 - MAE: 5.1495, RMSE: 7.2557
Fold 2, Epoch 1: Train Loss 240.4557, Test Loss 201.5645
Fold 2, Epoch 2: Train Loss 182.4447, Test Loss 120.1365
Fold 2, Epoch 3: Train Loss 121.9679, Test Loss 113.8813
Fold 2, Epoch 4: Train Loss 119.8112, Test Loss 111.1556
Fold 2, Epoch 5: Train Loss 116.7142, Test Loss 110.3356
Fold 2, Epoch 6: Train Loss 115.5272, Test Loss 108.1941
Fold 2, Epoch 7: Train Loss 114.0943, Test Loss 106.8793
Fold 2, Epoch 8: Train Loss 112.1882, Test Loss 106.4357
Fold 2, Epoch 9: Train Loss 110.4545, Test Loss 101.2380
Fold 2, Epoch 10: Train Loss 105.7330, Test Loss 91.6079
Fold 2, Epoch 11: Train Loss 92.5636, Test Loss 78.0291
Fold 2, Epoch 12: Train Loss 81.9776, Test Loss 70.0950
Fold 2, Epoch 13: Train Loss 79.5420, Test Loss 69.5266
Fold 2, Epoch 14: Train Loss 78.8071, Test Loss 69.0139
Fold 2, Epoch 15: Train Loss 76.7233, Test Loss 66.7565
Fold 2, Epoch 16: Train Loss 75.2259, Test Loss 67.7580
Fold 2, Epoch 17: Train Loss 75.6257, Test Loss 65.8087
Fol

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/2435093727.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 2 - MAE: 5.2849, RMSE: 7.6729
Fold 3, Epoch 1: Train Loss 244.4514, Test Loss 204.9531
Fold 3, Epoch 2: Train Loss 179.4778, Test Loss 124.1562
Fold 3, Epoch 3: Train Loss 122.1479, Test Loss 116.6795
Fold 3, Epoch 4: Train Loss 118.1492, Test Loss 112.0637
Fold 3, Epoch 5: Train Loss 116.4660, Test Loss 110.6533
Fold 3, Epoch 6: Train Loss 115.5651, Test Loss 109.2712
Fold 3, Epoch 7: Train Loss 113.8539, Test Loss 106.4856
Fold 3, Epoch 8: Train Loss 111.9072, Test Loss 104.3016
Fold 3, Epoch 9: Train Loss 107.1763, Test Loss 96.7952
Fold 3, Epoch 10: Train Loss 90.7012, Test Loss 71.8133
Fold 3, Epoch 11: Train Loss 80.8075, Test Loss 68.3148
Fold 3, Epoch 12: Train Loss 78.5638, Test Loss 72.4981
Fold 3, Epoch 13: Train Loss 77.2641, Test Loss 65.8621
Fold 3, Epoch 14: Train Loss 77.4474, Test Loss 66.3391
Fold 3, Epoch 15: Train Loss 75.3180, Test Loss 66.1287
Fold 3, Epoch 16: Train Loss 74.9122, Test Loss 64.5300
Fold 3, Epoch 17: Train Loss 75.1789, Test Loss 64.2738
Fold 

...

Fold 3 - MAE: 5.4974, RMSE: 7.7234
Fold 4, Epoch 1: Train Loss 241.5352, Test Loss 207.0091
Fold 4, Epoch 2: Train Loss 204.5371, Test Loss 156.8512
Fold 4, Epoch 3: Train Loss 126.7606, Test Loss 116.3864
Fold 4, Epoch 4: Train Loss 117.0100, Test Loss 112.3857
Fold 4, Epoch 5: Train Loss 113.9442, Test Loss 111.4023
Fold 4, Epoch 6: Train Loss 113.1145, Test Loss 109.2485
Fold 4, Epoch 7: Train Loss 111.5193, Test Loss 107.1660
Fold 4, Epoch 8: Train Loss 109.1143, Test Loss 111.7974
Fold 4, Epoch 9: Train Loss 106.8380, Test Loss 103.8346
Fold 4, Epoch 10: Train Loss 105.3968, Test Loss 102.4794
Fold 4, Epoch 11: Train Loss 101.1129, Test Loss 90.7004
Fold 4, Epoch 12: Train Loss 85.2256, Test Loss 74.6668
Fold 4, Epoch 13: Train Loss 80.2596, Test Loss 73.7686
Fold 4, Epoch 14: Train Loss 77.8933, Test Loss 70.3764
Fold 4, Epoch 15: Train Loss 75.8752, Test Loss 68.4138
Fold 4, Epoch 16: Train Loss 75.0752, Test Loss 68.3875
Fold 4, Epoch 17: Train Loss 73.7213, Test Loss 68.9804
F

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/2435093727.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 4 - MAE: 5.4267, RMSE: 7.7720
Fold 5, Epoch 1: Train Loss 233.7148, Test Loss 221.2677
Fold 5, Epoch 2: Train Loss 178.2826, Test Loss 135.5777
Fold 5, Epoch 3: Train Loss 117.7179, Test Loss 127.4925
Fold 5, Epoch 4: Train Loss 113.8052, Test Loss 125.5883
Fold 5, Epoch 5: Train Loss 112.5535, Test Loss 125.6906
Fold 5, Epoch 6: Train Loss 111.0757, Test Loss 122.6900
Fold 5, Epoch 7: Train Loss 110.7365, Test Loss 121.7621
Fold 5, Epoch 8: Train Loss 109.0269, Test Loss 119.3967
Fold 5, Epoch 9: Train Loss 106.5963, Test Loss 116.9749
Fold 5, Epoch 10: Train Loss 103.9335, Test Loss 114.6007
Fold 5, Epoch 11: Train Loss 93.1312, Test Loss 91.7045
Fold 5, Epoch 12: Train Loss 78.6106, Test Loss 92.0234
Fold 5, Epoch 13: Train Loss 75.0673, Test Loss 83.6682
Fold 5, Epoch 14: Train Loss 73.8673, Test Loss 82.2251
Fold 5, Epoch 15: Train Loss 72.0047, Test Loss 81.0272
Fold 5, Epoch 16: Train Loss 70.4805, Test Loss 81.6624
Fold 5, Epoch 17: Train Loss 70.4074, Test Loss 79.4798
Fo

/var/folders/bg/rj2597hd4399k2ht0hpblk9r0000gn/T/ipykernel_90635/2435093727.py:152: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'model_fo

Fold 5 - MAE: 5.5080, RMSE: 8.4807

Average MAE: 5.3733, Average RMSE: 7.7809
